# Various performance tips and helpful tools

## Code inspection macros

We can inspect the steps in the compilation process of a given function call all the way down to LLVM, thanks to a couple of macros. These can come in really handy when debugging performance issues.

In [1]:
function foo(x)
    x^2 + 3x - 1
end

foo (generic function with 1 method)

In [2]:
foo(1)

3

In [3]:
@code_lowered foo(1)

CodeInfo(
1 ─ %1 = Main.:^
│   %2 = Core.apply_type(Base.Val, 2)
│   %3 = (%2)()
│   %4 = Base.literal_pow(%1, x, %3)
│   %5 = 3 * x
│   %6 = %4 + %5
│   %7 = %6 - 1
└──      return %7
)

In [4]:
@code_typed foo(1)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(3, x)::Int64
│   %3 = Base.add_int(%1, %2)::Int64
│   %4 = Base.sub_int(%3, 1)::Int64
└──      return %4
) => Int64

In [5]:
@code_warntype foo(1)

MethodInstance for foo(::Int64)
  from 

foo(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(foo)


  x::Int64
Body::Int64


1 ─ %1 = Main.:^::Core.Const(^)


│   %2 = Core

.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %3 = (

%2)()::Core.Const(Val{2}())
│   %4 = Base.literal_pow(%1, x, %3)::Int64
│   %5 = (3 * x)::Int64
│   %6 = (%4 + %5)::Int64
│   %7 = (%6 - 1)::Int64
└──      return %7



In [6]:
@code_llvm foo(1)

;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define 

i64 @julia_foo_2112(i64 signext %0) #0 {
top:
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
; ┌ @ int.jl:87 within `+`
   %1 = add i64 %0, 3
   %2 = mul i64 %1, %0
; └
; ┌ @ int.jl:86 within `-`
   %3 = add i64 %2, -1
; └
  ret i64 %3
}


## Array views
When modifying arrays slice by slice, it's often handy to use `views` which are essentially "sub-arrays" that are directly bound to the same underlying memory, while having adapted indices.

In [7]:
function inplace!(a)
    for i in eachindex(a)
        a[i] = a[i]^2
    end
end

inplace! (generic function with 1 method)

In [8]:
x = rand(5)

5-element Vector{Float64}:
 0.8970701342338333
 0.6880062263744662
 0.41043656748222956
 0.8026228182589619
 0.779108820393122

In [9]:
inplace!(x)
x

5-element Vector{Float64}:
 0.8047348257343077
 0.4733525675300332
 0.1684581759265948
 0.6442033883899585
 0.6070105540143621

In [10]:
A = rand(5,5)

5×5 Matrix{Float64}:
 0.414202   0.634016  0.831322  0.707551  0.27304
 0.898552   0.474419  0.521679  0.834462  0.0648546
 0.518791   0.477753  0.928582  0.607521  0.615372
 0.371613   0.286019  0.932319  0.232151  0.846731
 0.0870965  0.799271  0.645996  0.468894  0.790626

In [11]:
for i in axes(A,1)
    inplace!(A[i,:])
end
A

5×5 Matrix{Float64}:
 0.414202   0.634016  0.831322  0.707551  0.27304
 0.898552   0.474419  0.521679  0.834462  0.0648546
 0.518791   0.477753  0.928582  0.607521  0.615372
 0.371613   0.286019  0.932319  0.232151  0.846731
 0.0870965  0.799271  0.645996  0.468894  0.790626

`A` hasn't changed! That's because `A[i,:]` creates a copy.

In order to get the correct behavior, we need to use views.

In [12]:
for i in axes(A,1)
    inplace!(view(A, i, :))
end
A

5×5 Matrix{Float64}:
 0.171563   0.401976   0.691097  0.500628   0.074551
 0.807395   0.225074   0.272149  0.696327   0.00420612
 0.269144   0.228248   0.862265  0.369082   0.378682
 0.138096   0.0818069  0.869218  0.0538939  0.716953
 0.0075858  0.638834   0.417311  0.219861   0.62509

Equivalently, we can use the `@views` macro, which will apply to every indexing operation within a block

In [13]:
 @views for i in axes(A,1)
    inplace!(A[i,:])
 end
 A

5×5 Matrix{Float64}:
 0.029434    0.161585    0.477614   0.250628    0.00555785
 0.651887    0.0506581   0.0740651  0.484871    1.76914e-5
 0.0724385   0.0520971   0.743501   0.136221    0.1434
 0.0190706   0.00669237  0.75554    0.00290455  0.514022
 5.75444e-5  0.408109    0.174148   0.048339    0.390738

## Type Stability

A function is said to be *type-stable* if its return type can be inferred from the types of its arguments. Type stability is pretty important, as its absence forces the Julia compiler to be more conservative. Any type unstable function will in general hurt performance, so they should be avoided at all costs.

The following function is type unstable, as the type of its output depends on the *value* of its input, not just its type.

In [14]:
function type_unstable(x)
    if x < 0
        return "Negative number"
    else
        return x
    end
end

type_unstable (generic function with 1 method)

A helpful tool to hunt for type instability is the `@code_warntype` macro.

In [15]:
@code_warntype type_unstable(1)

MethodInstance for type_unstable(::Int64)
  from type_unstable(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(type_unstable)
  x::Int64
Body

::Union{Int64, String}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return "Negative number"
3 ─      return x



A more subtle example of type instability is when handling arithmetic expression. Julia tends to avoid implicit type conversions, which can sometimes lead to surprises. The following function is type unstable. Can you spot why?

In [16]:
relu(x) = x < 0 ? 0 : x

relu (generic function with 1 method)

In [17]:
relu(1.0), relu(-1.0)

(1.0, 0)

In [18]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(relu)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



The problem here is that the ternary operator returns 0 of type `Int` regardless of the type of `x`. In order to correct this, we can use the function `zero` which will return the zero value of the same type as `x`. Other functions like that include `one(x)` and `oftype(x,y)`

In [19]:
relu(x) = x < 0 ? zero(x) : x

relu (generic function with 1 method)

In [20]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(relu)
  x::Float64
Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



Another common issue with types is when incorrectly using abstract types. For example, the following type definition is particularly problematic

In [21]:
struct BadFoo
    x # this amounts to x::Any
    y::Real
    z::Vector{Integer}
end

Why is this bad? Well, because whenever a struct has abstract field types, the Julia compiler can't tell what it will hold, so it has to be ready for anything, which means using pointers to values, rather than the values themselves.

The correct way to do this is to use a parametric type, so that for any particular instance of our type, the compiler knows exactly the types of each field.

In [22]:
struct GoodFoo{T1,T2<:Real,T3<:Integer}
    x::T1
    y::T2
    z::Vector{T3}
end

As an example of how bad abstract field types (or element types can be) let's look at the generated code with `@code_llvm`.

In [23]:
mutable struct Bar{T<:AbstractFloat}
    a::T
end

In [24]:
foo(b::Bar) = m.a + 1

foo (generic function with 2 methods)

In [25]:
code_llvm(foo, Tuple{Float64})

;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define double @julia_foo_2666(double %0) #0 {
top:
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
; ┌ @ intfuncs.jl:319 within `literal_pow`
; │┌ @ float.jl:410 within `*`
    %1 = fmul double %0, %0
; └└
; ┌ @ promotion.jl:411 within `*` @ float.jl:410
   %2 = fmul double %0, 3.000000e+00
; └
; ┌ @ float.jl:408 within `+`
   %3 = fadd double %1, %2
; └
; ┌ @ promotion.jl:412 within `-` @ float.jl:409
   %4 = fadd double %3, -1.000000e+00
; └
  ret double %4
}


In [26]:
code_llvm(foo, Tuple{AbstractFloat})

; WARNING: This code may not match what actually runs.
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define nonnull {}* @julia_foo_2668({}* noundef nonnull readonly %0) #0 {
top:
  %1 = alloca [3 x {}*], align 8
  %gcframe5 = alloca [4 x {}*], align 16
  %gcframe5.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe5, i64 0, i64 0
  %.sub = getelementptr inbounds [3 x {}*], [3 x {}*]* %1, i64 0, i64 0
  %2 = bitcast [4 x {}*]* %gcframe5 to i8*
  call void @llvm.memset.p0i8.i32(i8* noundef nonnull align 16 dereferenceable(32) %2, i8 0, i32 32, i1 false)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"() #5
  %ppgcstack_i8 = getelementptr i8, i8* %thread_ptr, i64 -8
  %ppgcstack = bitcast i8* %ppgcstack_i8 to {}****
  %pgcstack = load {}***, {}**** %ppgcstack, align 8
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
  %3 = bitcast [4 x {}*]* %gcframe5 to i64*
  store i64 8, i64* %3, align 16


*), {}** %8, align 8
  %22 = call nonnull {}* @ijl_apply_generic({}* inttoptr (i64 140308617666720 to {}*), {}** nonnull %.sub, i32 2)
  %23 = load {}*, {}** %4, align 8
  %24 = bitcast {}*** %pgcstack to {}**
  store {}* %23, {}** %24, align 8
  ret {}* %22
}


Notice how the second one is so much longer? That's because of all the checks the compiler has to do because it's unsure about the type. This, by the way is how Python operates by default.